In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.download.n

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-Vine-Review").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", 'verified_purchase'])

In [ ]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [ ]:
vote_total = vine_df.filter(vine_df.total_votes>=20)

In [93]:
#total number of products that have 20 or more votes
total_count = vote_total.count()
total_count

67855

In [ ]:
vote_total.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [40]:
from pyspark.sql.functions import round,concat, col, lit


In [41]:
helpful_votes_percentage_df = vote_total.withColumn('helpful_votes_percentage' , round(vote_total['helpful_votes']/vote_total['total_votes']*100,2))


In [38]:
helpful_votes_percentage_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  %|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+---+------------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|  %|                   100.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|  %|                   100.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|  %|                    80.0|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|  %|                   96.88|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|  %|                   73.85|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|  %|                   93.26|
|R36LSV4WQ4VB1K|          5|         

In [42]:
helpful_votes_percentage_df.select(concat(col('helpful_votes_percentage'),lit('%')))

DataFrame[concat(helpful_votes_percentage, %): string]

In [77]:
helpful_votes_df = helpful_votes_percentage_df.select('*',concat(col('helpful_votes_percentage'),col('%')).alias('Percentage_helpful'))

In [78]:
helpful_votes_df_2 = helpful_votes_df.drop('%', 'helpful_votes_percentage')

In [79]:
helpful_votes_df_2.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|            100.0%|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|            100.0%|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|             80.0%|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|            96.88%|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|            73.85%|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|            93.26%|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|            77.27%|
|R18BTOE5UGN7UA|          5|  

In [80]:
fifty_percent_df = helpful_votes_df_2.filter(helpful_votes_percentage_df.helpful_votes_percentage>50.00)

In [81]:
fifty_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|            100.0%|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|            100.0%|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|             80.0%|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|            96.88%|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|            73.85%|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|            93.26%|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|            77.27%|
|R18BTOE5UGN7UA|          5|  

In [95]:
fifty_percent_count = fifty_percent_df.count()
fifty_percent_count

61807

In [ ]:
fifty_percent_df = fifty_percent_df.filter

In [82]:
paid_review_df = fifty_percent_df.filter(fifty_percent_df.vine =='Y')

In [83]:
paid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|             87.1%|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|            76.19%|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|             87.8%|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|            97.33%|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|            100.0%|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|            100.0%|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|             95.0%|
|R1803HITBY7Y08|          4|  

In [84]:
unpaid_review_df = fifty_percent_df.filter(fifty_percent_df.vine =='N')

In [85]:
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|            100.0%|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|            100.0%|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|             80.0%|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|            96.88%|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|            73.85%|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|            93.26%|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|            77.27%|
|R18BTOE5UGN7UA|          5|  

In [86]:
# number of paid reviews that are 5-stars
paid_fivestar_count = paid_review_df.filter(paid_review_df.star_rating==5).count()
paid_fivestar_count

139

In [87]:
# number of unpaid reviews that are 5-stars
unpaid_fivestar_count = unpaid_review_df.filter(paid_review_df.star_rating==5).count()
unpaid_fivestar_count

32646

In [73]:
# total number of vine reivews
total_paid_reviews = paid_review_df.count()
total_paid_reviews

333

In [98]:
# percentage of five star vine reviews 
percentage_paid = (paid_fivestar_count/total_paid_reviews)*100
percentage_paid

41.74174174174174

In [75]:
#total number of non-vine reviews
total_unpaid_reviews = unpaid_review_df.count()
total_unpaid_reviews

61474

In [99]:
percentage_unpaid= (unpaid_fivestar_count/total_unpaid_reviews)*100
percentage_unpaid

53.10537788333279